#Library Import

In [ ]:
from keras.models import Sequential

import tensorflow as tf

import tensorflow_datasets as tfds

from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, RMSprop, Adam
# !pip install np_utils
from keras.utils import to_categorical
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import random
from numpy import *
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Preparing our dataset for training will involve assigning paths and creating categories(labels), resizing our images.

# Resizing images into 200 X 200

In [ ]:
path_test = "/content/drive/MyDrive/Colab Notebooks/Cell Type Classification/images/TRAIN"

CATEGORIES = ["EOSINOPHIL", "LYMPHOCYTE", "MONOCYTE", "NEUTROPHIL"]
# print(img_array.shape)
IMG_SIZE =200
# new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))

# Training is an array that will contain image pixel values and the index at which the image in the CATEGORIES list.

In [ ]:
training = []
img_array = []
def createTrainingData():
  for category in CATEGORIES:
    path = os.path.join(path_test, category)
    class_num = CATEGORIES.index(category)
    # print(os.listdir(path))
    for img in os.listdir(path):
      img_array = cv2.imread(os.path.join(path,img))
      new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
      training.append([new_array, class_num])

createTrainingData()


In [ ]:
#  print(training[1])

In [ ]:
random.shuffle(training)

#Assigning Labels and Features

In [ ]:
X =[]
y =[]
for features, label in training:
  X.append(features)
  y.append(label)
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y= np.array(y)

# Normalising X and Converting Labels to Categorical Data

In [ ]:
X = X.astype('float32')
X /= 255
# from keras.utils import np_utils
Y = to_categorical(y, 4)
print(Y[100])
print(shape(Y))

[0. 0. 0. 1.]
(8331, 4)


# Split X and Y for Use in CNN

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 4)

# Define, Compile and Train the CNN Model

In [ ]:
batch_size = 16
nb_classes =4
nb_epochs = 5
img_rows, img_columns = 200, 200
img_channel = 3
nb_filters = 32
nb_pool = 2
nb_conv = 3

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu,
                           input_shape=(200, 200, 3)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(4,  activation=tf.nn.softmax)
])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train, y_train, batch_size = batch_size, epochs = nb_epochs, verbose = 1, validation_data = (X_test, y_test))

# Accuracy and Score of Model

In [ ]:
score = model.evaluate(X_test, y_test, verbose = 0 )
print("Test Score: ", score[0])
print("Test accuracy: ", score[1])